In [ ]:
# stdlib
import os

# syft absolute
import syft as sy

# os.environ["ORCHESTRA_DEPLOYMENT_TYPE"] = "remote"
# os.environ["CLUSTER_HTTP_PORT_HIGH"] = "9081"
# os.environ["CLUSTER_HTTP_PORT_LOW"] = "9083"

In [ ]:
environment = os.environ.get("ORCHESTRA_DEPLOYMENT_TYPE", "python")
high_port = os.environ.get("CLUSTER_HTTP_PORT_HIGH", "auto")
low_port = os.environ.get("CLUSTER_HTTP_PORT_LOW", "auto")

# Launch server and login

In [ ]:
server_low = sy.orchestra.launch(
    name="bigquery-low",
    server_side_type="low",
    dev_mode=True,
    n_consumers=1,
    create_producer=True,
    port=low_port,
)

server_high = sy.orchestra.launch(
    name="bigquery-high",
    server_side_type="high",
    dev_mode=True,
    n_consumers=1,
    create_producer=True,
    port=high_port,
)

In [ ]:
low_client = server_low.login(email="info@openmined.org", password="changethis")
high_client = server_high.login(email="info@openmined.org", password="changethis")

In [ ]:
# syft absolute
from syft.client.syncing import compare_clients

In [ ]:
# syft absolute
from syft.service.job.job_stash import Job
from syft.service.job.job_stash import JobStatus

# Sync UserCode and Requests to High Side

In [ ]:
def is_job_to_sync(batch):
    if batch.status != "NEW":
        return False
    if not isinstance(batch.root.high_obj, Job):
        return False
    job = batch.root.high_obj
    return job.status in (JobStatus.ERRORED, JobStatus.COMPLETED)

In [ ]:
def sync_new_objects(
    from_client, to_client, dry_run: bool = True, private_data: bool = False
):
    sim = "Simulating " if dry_run else ""
    priv = "WITH PRIVATE DATA" if private_data else ""
    print(f"{sim}Syncing from {from_client.name} to {to_client.name} {priv}")
    changes = []
    diff = compare_clients(
        from_client=from_client, to_client=to_client, hide_usercode=False
    )
    if isinstance(diff, sy.SyftError):
        return diff

    for batch in diff.batches:
        try:
            if is_job_to_sync(batch) or batch.status == "NEW":
                w = batch.resolve(build_state=False)
                if private_data:
                    w.click_share_all_private_data()
                if not dry_run:
                    w.click_sync()
                change_text = f"Synced {batch.status} {batch.root_type.__name__}"
                if not dry_run:
                    changes.append(change_text)
                else:
                    print(f"Would have run: {change_text}")
        except Exception as e:
            print("sync_new_objects", e)
            raise e
    return changes

In [ ]:
diffs = compare_clients(
    from_client=low_client, to_client=high_client, hide_usercode=False
)

In [ ]:
# check that only requests and usercode are in the diff
assert {diff.root_diff.obj_type.__qualname__ for diff in diffs.batches} == {
    "Request",
    "UserCode",
}

In [ ]:
widget = diffs.resolve()

widget._share_all()
widget._sync_all()

# Check that request synced over to high side

In [ ]:
assert len(high_client.code.get_all()) == 2

In [ ]:
requests = high_client.requests
requests

In [ ]:
user_request = None
for request in requests:
    if "large_sample" in getattr(
        getattr(request, "code", None), "service_func_name", None
    ):
        user_request = request
assert user_request

# Execute Request

In [ ]:
job = high_client.code.large_sample(blocking=False)

In [ ]:
# third party
import pandas as pd

res = job.wait().get()

assert isinstance(res, pd.DataFrame)

# Sync job result to low side

In [ ]:
diffs = compare_clients(
    from_client=high_client, to_client=low_client, hide_usercode=False
)

In [ ]:
assert len(diffs.batches) == 1
assert diffs.batches[0].root_diff.obj_type.__qualname__ == "Job"

In [ ]:
widget = diffs.resolve()

widget._share_all()
widget._sync_all()

# Check that job was synced to low side

In [ ]:
requests = low_client.requests
requests

In [ ]:
user_request = None
for request in requests:
    if "large_sample" in getattr(
        getattr(request, "code", None), "service_func_name", None
    ):
        user_request = request

In [ ]:
assert "approved" in str(user_request.status).lower()

In [ ]:
if environment != "remote":
    server_high.land()
    server_low.land()